# Live Demo
## Bibliotheken

In [1]:
from Research_Scraper_Code.Research_Scraper import ResearchScraper
from Research_Scraper_Code import utils
import pandas as pd
import json
import os
from scholarly import scholarly, ProxyGenerator
from scholarly import MaxTriesExceededException

In [2]:
scraper = ResearchScraper()

## BS Demo

In [3]:
sample_dois = ['10.1007/s12525-020-00445-0',
               '10.1002/joom.1125',
               '10.1007/978-3-030-49570-1_14',
               '10.1016/j.jvcir.2018.09.015',
               '10.1109/MIS.2017.40']

Diese Liste wird jetzt gescraped

In [4]:
def demo_scrape():
    scraping_results = scraper.scrape_publication_by_doi_list(sample_dois, params=['full'])
    print(f'Anzahl der Ergebnisse: {len(scraping_results)}')
    return scraping_results

scraping_results = demo_scrape()

Time of scrape start: 2022_10_27__09_18
>>> Scraping 10.1007/s12525-020-00445-0 #0
URL ('https://doi.org/10.1007/s12525-020-00445-0') is a DOI link, Links is now resolved properly
Resolved DOI link to: https://link.springer.com/article/10.1007/s12525-020-00445-0
[DEBUG - ResearchScraper] - Found scraper for https://link.springer.com/article/10.1007/s12525-020-00445-0 -> ScraperSpringer
 Scraped keys: ['url', 'title', 'doi', 'authors', 'keywords', 'abstract', 'pdf', 'publisher', 'year', 'start_page', 'end_page', 'publication_type', 'full_text', 'references', 'journal_name', 'journal_volume', 'article_accesses', 'amount_citations']
>>>> Scraping 10.1007/s12525-020-00445-0 done
>>>> Scraping 10.1007/s12525-020-00445-0 added to results
>>> Scraping 10.1002/joom.1125 #1
URL ('https://doi.org/10.1002/joom.1125') is a DOI link, Links is now resolved properly
Resolved DOI link to: https://onlinelibrary.wiley.com/doi/10.1002/joom.1125
[DEBUG - ResearchScraper] - No scraper found for https://onl

## PDF Demo
CRIS einlesen

In [6]:
def load_publications_from_csv():
    data = 'data/publications_without_abstract.csv'

    with open(data) as f:
        df = pd.read_csv(f, sep=';')
    return df

Cleaning

In [7]:
df_publications = load_publications_from_csv()
df_publications_with_url = df_publications[df_publications['url'].notna()]
df_publications_with_url = df_publications_with_url[df_publications_with_url['url'].str.endswith('.pdf')]
df_publications_with_url = df_publications_with_url[['id', 'url']]
# only first 10 entries
df_publications_with_url = df_publications_with_url.head(50)
print(f'There are {len(df_publications_with_url)} publications with a pdf url')

There are 50 publications with a pdf url


In [8]:
def download_pdf_from_cris(df_publications):
    df_prepared = df_publications[df_publications['url'].notna()]
    df_prepared = df_prepared[df_prepared['url'].str.endswith('.pdf')]
    df_prepared = df_prepared[['id', 'url']]
    df_prepared = df_prepared.reset_index(drop=True)
    print(f'shape of df_prepared: {df_prepared.shape}')

    folder = 'exports/pdf_downloads/live_demo'
    for index, row in df_prepared.iterrows():
        print(f'Downloading {index + 1} of {len(df_prepared)} publications')
        filename = f'document_{row["id"]}'
        pdf_url = row['url']
        utils.download_pdf(url=pdf_url, filename=filename, write_folder_path=folder, timeout=2)


download_pdf_from_cris(df_publications_with_url)

shape of df_prepared: (50, 2)
[utils.py: download_pdf] Connection Error - could not download: error type: <class 'requests.exceptions.ConnectionError'> -> HTTPConnectionPool(host='udoo.uni-muenster.de', port=80): Max retries exceeded with url: /downloads/publications/2011.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fac19854c40>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))
[utils.py: download_pdf] Connection Error - could not download: error type: <class 'requests.exceptions.ConnectTimeout'> -> HTTPConnectionPool(host='web.up.ac.za', port=80): Max retries exceeded with url: /ecis/ECIS2010PR/ECIS2010/Content/Papers/0451.R1.pdf (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x7fac29105bb0>, 'Connection to web.up.ac.za timed out. (connect timeout=2)'))
[utils.py: download_pdf] Connection Error - could not download: error type: <class 'requests.exceptions.ConnectionError'>